<a href="https://colab.research.google.com/github/sivakumarAI/MachineLearning/blob/master/CreditScore_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb



In [72]:
from google.colab import files
uploaded_train = files.upload()

Saving CreditScore_train.csv to CreditScore_train (2).csv


In [74]:
from google.colab import files
uploaded_test = files.upload()

Saving CreditScore_test.csv to CreditScore_test (1).csv


In [0]:
MSE_Train = []
RMSE_Train = []
MAE_Train = []
MSE_Test = []
RMSE_Test = []
MAE_Test = []
Hyper =[]
rfestimator = []
tree_depth = []
MAPE_Train = []
MAPE_Test = []
    

In [85]:
ModelFlow()

KeyError: ignored

In [0]:
#Execution Flow

def ModelFlow():
  
  import io
  CreditTrain = pd.read_csv(io.BytesIO(uploaded_train['CreditScore_train (1).csv']))
  CreditTest = pd.read_csv(io.BytesIO(uploaded_test['CreditScore_test.csv']))
                  
    
  #Concatenate both train and test data
  Credit_Train_Test = pd.concat([CreditTrain,CreditTest],axis=0)
  #print("CreditTrain:",CreditTrain.head())
  #print("CreditTest:",CreditTest.head())
  #print("Credit_Train_Test:",Credit_Train_Test.head())

  #Know the data
  KnowTheData(Credit_Train_Test)
    
  print("**********************************************")
  print("Linear Model Type Build Process starts.....")
  Credit_Train_Test_Linear = ModelType(Credit_Train_Test,modeltp = "linear")
            
  #Split the dataset into train and test after datacleaning and selected the important features for the model.
  Credit_Train = Credit_Train_Test_Linear.head(80000)
  Credit_Test = Credit_Train_Test_Linear.tail(20000)
    
  print("Model Selection Starts for Linear.....")
  #Model_Selection
  X_train,X_test,Y_train,Y_test, Credit_Test = ModelSelection(Credit_Train, Credit_Test)
  print("Model Selection Ends for Linear.....")
    
  #Build Linear Model
  LinearReg(X_train,X_test,Y_train,Y_test,Credit_Test)
   
  #Build Ridge Model
  RidgeReg(X_train,X_test,Y_train,Y_test,Credit_Test)
    
  print("Linear Model Type Build Process ends.....")
  print("**********************************************")
    
  print("**********************************************")
  print("Tree Model Type Build Process starts.....")
       
  Credit_Train_Test_Tree = ModelType(Credit_Train_Test,modeltp = "Tree")
    
  #Split the dataset into train and test after datacleaning and selected the important features for the model.
  Credit_Train = Credit_Train_Test_Tree.head(80000)
  Credit_Test = Credit_Train_Test_Tree.tail(20000)
    
  print("Model Selection Starts for Tree.....")
  #Model_Selection
  X_train,X_test,Y_train,Y_test, Credit_Test = ModelSelection(Credit_Train, Credit_Test)
    
  print("Model Selection Ends for Tree.....")
    
  #Build DSRegressor Model
  DSReg(X_train,X_test,Y_train,Y_test,Credit_Test)
    
  #Build RandomForest Regressor Model
  RandomForestReg(X_train,X_test,Y_train,Y_test,Credit_Test)
    
  #Build XGBoost Regressor Model
  XGBoost(X_train,X_test,Y_train,Y_test,Credit_Test)
    
  print("**********************************************")
  print("Tree Model Type Build Process ends.....")
    

In [0]:
#Knowing the data
def KnowTheData(Credit_Train_Test):
    display("Shape of the dataframe:")
    display(Credit_Train_Test.shape)
    CreditTrain_data = []
    CreditTrain_data.append(['FeatureName','DataType','No.ofMiss_Values','Percent_of_Miss_Values'])
    for col in Credit_Train_Test.columns:
        CreditTrain_data.append([col,Credit_Train_Test[col].dtypes,Credit_Train_Test[col].isnull().sum(),(Credit_Train_Test[col].isnull().sum() / len(Credit_Train_Test) * 100)])
    
    #display(CreditTrain_data)
    


In [0]:
def ModelType(Credit_Train_Test,modeltp):
    if modeltp == "linear":
        Credit_Train_Test = DataCleaning(Credit_Train_Test,modeltp)
        Credit_Train_Test = Feature_Selection(Credit_Train_Test)
    else:
        Credit_Train_Test = DataCleaning(Credit_Train_Test,modeltp)
    
    return Credit_Train_Test
        

In [0]:
def DataCleaning(Credit_Train_Test,modeltp):
    DuplicatedData(Credit_Train_Test)
    if modeltp == "linear":
        print("DataCleaning Process Starts For Linear.....")
        Credit_Train_Test = DataForImputation(Credit_Train_Test,modeltp)
        Credit_Train_Test = Imputation(Credit_Train_Test,modeltp)
        print("DataCleaning Process Ends For Linear.....")
        
    else:
        print("DataCleaning Process Starts For Tree.....")
        Credit_Train_Test = Imputation(Credit_Train_Test,modeltp)
        print("DataCleaning Process Ends For Tree.....")
        
    return Credit_Train_Test

In [0]:
def DuplicatedData(Credit_Train_Test):
    print("Finding Duplicated Data")
    print("Total number of duplicate records:", Credit_Train_Test.duplicated().sum())

In [0]:
def DataForImputation(Credit_Train_Test,modeltp):
    
    Features_For_Model = []
    Missing_count = 0.0
        
    if modeltp == 'linear':
        
        print("DataAnalysis For Imputation Starts For Linear...")
        print("Drop the features whose greater than equal to 50% of Missing Values:")
        
        for col in Credit_Train_Test.columns:
            
            Missing_count = (Credit_Train_Test[col].isnull().sum()/ len(Credit_Train_Test)) * 100
            if Missing_count < 50:
                Features_For_Model.append(col)
            else:
                
                #display(Credit_Train_Test[col])
                Credit_Train_Test.drop(Credit_Train_Test[[col]],axis=1,inplace=True)
    
    #display(np.array(Features_For_Model).T)
    print("DataAnalysis For Imputation Ends For Linear...")
    
    return Credit_Train_Test
    
        

In [0]:
#Impute Missing Values:

def Imputation(Credit_Train_Test,modeltp):
    
       
    if modeltp == "linear":
        print("Imputation starts for Linear.....")
        for col in Credit_Train_Test.columns:
            Credit_Train_Test[col].fillna(Credit_Train_Test[col].mean(),inplace=True)
        print("Shape of the Dataframe after Imputation for Linear:")
        display(Credit_Train_Test.shape)
        print("Imputation ends for Linear.....")
        
    else:
        print("Imputation starts for Tree.....")
        display(Credit_Train_Test.shape)
        Credit_Train_Test.fillna(0,inplace=True)
        print("Imputation ends for Tree.....")
    
    return Credit_Train_Test
        
    

In [0]:
def Feature_Selection(Credit_Train_Test):
    print("Identifying the relationship of independent and dependent variables for Linear")
    Credit_Train_Test_Corr = Credit_Train_Test.corr(method='pearson')[['y']].T
    Credit_Train_Test_Corr = Credit_Train_Test_Corr[Credit_Train_Test_Corr > 0.3]
    #display(CreditTrain_Corr.columns)
    Important_Features = []
    for col in Credit_Train_Test_Corr.columns:
        if Credit_Train_Test_Corr[col].isnull()[0] != True:
            Important_Features.append(col)
                    
    print("Important Features from Correlation for Linear")
    #display(Important_Features)
    #display(Credit_Train_Test_Corr)
    #Multicolinearity(CreditTrain,Important_Features)  
    return Credit_Train_Test[Important_Features]
    

In [0]:
#def Multicolinearity(CreditTrain,Important_Features):
 #   display("Multicolinearity...")
  #  CreditTrain_Corr = CreditTrain[Important_Features].corr()
   # CreditTrain_Corr = CreditTrain_Corr[CreditTrain_Corr < 0.75]
    #HighMultiColinear_Features = []
    #display(CreditTrain_Corr)
    #for col1 in CreditTrain_Corr.columns:
     #   for col2 in CreditTrain_Corr.columns:
      #      if col1 != col2 and col1 != 'y' and col2!= 'y' and CreditTrain_Corr[col1][col2] >= 0.75:
       #         if CreditTrain_Corr[col1]['y'] >= CreditTrain_Corr[col2]['y']:
        #            if col1 not in HighMultiColinear_Features:
         #               HighMultiColinear_Features.append(col1)
                    
    #display(np.array(HighMultiColinear_Features).T)

In [0]:
def ModelSelection(CreditTrain,Credit_Test):
    print("Model Selection Starts.....")
    X = CreditTrain.drop('y',axis=1)
    Y = CreditTrain['y']
    Credit_Test.drop('y',axis=1,inplace=True)
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=10)
    
    #print("X_train shape:", X_train.shape)
    #print("X_test shape:", X_test.shape)
    #print("Y_train shape:", Y_train.shape)
    #print("Y_test shape:", Y_test.shape)
    print("Model Selection Ends.....")
    return X_train,X_test,Y_train,Y_test,Credit_Test
    

In [0]:
def LinearReg(X_train,X_test,Y_train,Y_test,Credit_Test):
    algo = 'linear'
    model = LinearRegression()
    model.fit(X_train,Y_train)
    print("LinearRegression:")
    Y_train_predict = model.predict(X_train)
    Y_test_predict = model.predict(X_test)
    Y_Credit_test_predict = model.predict(Credit_Test)
    print("********Linear Regression Measures****************")
    Measures(Y_train,Y_train_predict,Y_test,Y_test_predict,Y_Credit_test_predict)
    Linear_accuracy = pd.DataFrame({"MSE-Train":MSE_Train,"MSE-Test":MSE_Test,"RMSE-Train":RMSE_Train,"RMSE-Test":RMSE_Test,"MAE-Train":MAE_Train,"MAE-Test":MAE_Test,"MAPE-Train":MAPE_Train,"MAPE-Test":MAPE_Test})
    ModelPerformance(algo,Linear_accuracy)
    

In [0]:
def RidgeReg(X_train,X_test,Y_train,Y_test,Credit_Test):
    global Hyper
    algo = 'Ridge'
    
    print("RidgeRegression:")
    print("********Ridge Regression Measures****************")
    
    for alpha in range(1,10):
        model = Ridge(alpha,normalize=True)
        Hyper.append(alpha)
        model.fit(X_train,Y_train)
        Y_train_predict = model.predict(X_train)
        Y_test_predict = model.predict(X_test)
        Y_Credit_test_predict = model.predict(Credit_Test)
        Measures(Y_train,Y_train_predict,Y_test,Y_test_predict,Y_Credit_test_predict)
        
    Ridge_optimize = pd.DataFrame({"Penalty":Hyper,"MSE-Train":MSE_Train,"MSE-Test":MSE_Test,"RMSE-Train":RMSE_Train,"RMSE-Test":RMSE_Test,"MAE-Train":MAE_Train,"MAE-Test":MAE_Test,"MAPE-Train":MAPE_Train,"MAPE-Test":MAPE_Test})
        
    ModelPerformance(algo, Ridge_optimize)
    
    


In [0]:
def DSReg(X_train,X_test,Y_train,Y_test,Credit_Test):
    global Hyper
    algo = 'ds'
    print("Decision Tree:")
    print("********Decision Tree Regression Measures****************")
    for depth in range(1,20):
        dsmodel = tree.DecisionTreeRegressor(max_depth=depth)
        Hyper.append(depth)
        dsmodel.fit(X_train,Y_train)
        Y_train_predict = dsmodel.predict(X_train)
        Y_test_predict = dsmodel.predict(X_test)
        Y_Credit_test_predict = dsmodel.predict(Credit_Test)
        Measures(Y_train,Y_train_predict,Y_test,Y_test_predict,Y_Credit_test_predict)
    
    ds_optimize = pd.DataFrame({"TreeDepth":Hyper,"MSE-Train":MSE_Train,"MSE-Test":MSE_Test,"RMSE-Train":RMSE_Train,"RMSE-Test":RMSE_Test,"MAE-Train":MAE_Train,"MAE-Test":MAE_Test,"MAPE-Train":MAPE_Train,"MAPE-Test":MAPE_Test})
    
    ModelPerformance(algo,ds_optimize)
    

In [0]:
def RandomForestReg(X_train,X_test,Y_train,Y_test,Credit_Test):
    global rfestimator, tree_depth
    algo = "RandomForest"
    print("**************Random Forest Regressor*****************")
    
    for estimator in range(10,21,2):
        
        for depth in range(1,10,2):
            rfestimator.append(estimator)
            tree_depth.append(depth)
            random_model = RandomForestRegressor(n_estimators=estimator,max_depth = depth, random_state=0)
            random_model.fit(X_train,Y_train)
            Y_train_predict = random_model.predict(X_train)
            Y_test_predict = random_model.predict(X_test)
            Y_Credit_test_predict = random_model.predict(Credit_Test)
            
            Measures(Y_train,Y_train_predict,Y_test,Y_test_predict,Y_Credit_test_predict)
        
    random_accuracy = pd.DataFrame({"No of Trees":rfestimator,"Tree Depth": tree_depth,"MSE-Train":MSE_Train,"MSE-Test":MSE_Test,"RMSE-Train":RMSE_Train,"RMSE-Test":RMSE_Test,"MAE-Train":MAE_Train,"MAE-Test":MAE_Test,"MAPE-Train":MAPE_Train,"MAPE-Test":MAPE_Test})
    
    ModelPerformance(algo,random_accuracy)
    rfestimator = []
    tree_depth = []
    
    

In [0]:
def XGBoost(X_train,X_test,Y_train,Y_test,Credit_Test):
    print("**************XGBOOST**********************")
    global rfestimator, tree_depth
    algo = "XGBoost"
    estimator = 1000
    depth = 9
    #for estimator in range(100,1001,100):
        
     #   for depth in range(1,10):
    rfestimator.append(estimator)
    tree_depth.append(depth)
        #xgb_model = xgb.XGBRegressor(n_estimators=estimator, learning_rate=0.1, 
         #                        subsample=0.75,colsample_bytree=1,max_depth=depth,
          #                       random_state=10,gamma=1)
    xgb_model = xgb.XGBRegressor(objective ='reg:squarederror')

    xgb_model.fit(X_train,Y_train)
    Y_train_predict = xgb_model.predict(X_train)
    Y_test_predict = xgb_model.predict(X_test)
    Y_Credit_test_predict = xgb_model.predict(Credit_Test)
    Measures(Y_train,Y_train_predict,Y_test,Y_test_predict,Y_Credit_test_predict)
   
    xgb_accuracy = pd.DataFrame({"No of Trees":rfestimator,"Tree Depth": tree_depth,"MSE-Train":MSE_Train,"MSE-Test":MSE_Test,"RMSE-Train":RMSE_Train,"RMSE-Test":RMSE_Test,"MAE-Train":MAE_Train,"MAE-Test":MAE_Test,"MAPE-Train":MAPE_Train,"MAPE-Test":MAPE_Test})
        
    ModelPerformance(algo,xgb_accuracy)
    rfestimator = []
    tree_depth = []
       

In [0]:
def Measures(Y_train,Y_train_predict,Y_test,Y_test_predict,Y_Credit_test_predict):
    
    global MSE_Train, RMSE_Train,MAE_Train, MSE_Test, RMSE_Test, MAE_Test, MAPE_Train, MAPE_Test, Hyper
    
    MSE = mean_squared_error(Y_train,Y_train_predict)
    RMSE = np.sqrt(mean_squared_error(Y_train,Y_train_predict))
    MAE = mean_absolute_error(Y_train,Y_train_predict)
    MAPE = np.mean(np.abs((Y_train - Y_train_predict) / Y_train)) * 100

        
    MSE_Train.append(MSE)
    RMSE_Train.append(RMSE)
    MAE_Train.append(MAE)
    MAPE_Train.append(MAPE)
               
        
    MSE = mean_squared_error(Y_test,Y_test_predict)
    RMSE = np.sqrt(mean_squared_error(Y_test,Y_test_predict))
    MAE = mean_absolute_error(Y_test,Y_test_predict)
    MAPE = np.mean(np.abs((Y_test - Y_test_predict) / Y_test)) * 100
        
    MSE_Test.append(MSE)
    RMSE_Test.append(RMSE)
    MAE_Test.append(MAE)
    MAPE_Test.append(MAPE)
    
        
    

In [0]:
def ModelPerformance(algo,performance):
    print("Performance of the model:", algo)
    global Hyper, MSE_Train, RMSE_Train,MAE_Train,MSE_Test, RMSE_Test,MAE_Test,MAPE_Train,MAPE_Test
    
    display(performance)  
    if algo == 'XGBoost':
        plt.figure(figsize=(10,5))
        sns.lineplot(x=performance["No of Trees"],y=performance["MSE-Train"])
               
        sns.lineplot(x=performance["No of Trees"],y=performance["MSE-Test"])
        plt.xlabel("No of Trees")
        plt.ylabel("MSE-Error")
                
        plt.show()
        
    Hyper = []
    MSE_Train = []
    RMSE_Train = []
    MAE_Train = []
    MSE_Test = []
    RMSE_Test = []
    MAE_Test = []
    MAPE_Train = []
    MAPE_Test = []
    MAPE_Train = []
    MAPE_Test = []
    